In [36]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os 
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [37]:
load_dotenv()

True

In [38]:
key = os.getenv("OPENAI_API_KEY")

In [39]:
print(key)

sk-proj-on6KKAnxGqKMN1hdFVOLT3BlbkFJJBqK2seFj2RwjFeQbVEr


In [40]:
llm = ChatOpenAI(openai_api_key = key, model= 'gpt-3.5-turbo', temperature=0.7)

In [41]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000208F8DFB610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000208FA600290>, openai_api_key='sk-proj-on6KKAnxGqKMN1hdFVOLT3BlbkFJJBqK2seFj2RwjFeQbVEr', openai_proxy='')

In [42]:
with open(r"C:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\response.json") as f:
    RESPONSE_JSON = json.load(f)

In [43]:
print(RESPONSE_JSON)

{'1': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}, '2': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}, '3': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}, '4': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}}


In [44]:
TEMPLATE = """
Text : {text}
You are a expert in MCQ maker. Given the above text, it is your job to create quiz \
of {number} multiple choice question for subject in {tone}. Make sure the questions are not repeated and check all the questions to be conforming.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
ensure to make {number} MCQ

### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [45]:
quiz_generation_prompt=PromptTemplate(
    input_variables = ["text", "numbers", "tone", "response_json"],
    template=TEMPLATE
)

In [65]:
from langchain.chains import LLMChain

# Assuming `llm` and `quiz_generation_prompt` are already defined
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [66]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [67]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [68]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [69]:

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [82]:
PATH = r"C:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\data.txt"

with open(PATH, 'r', encoding='utf-8') as file:
    TEXT = file.read()


In [83]:
TEXT

'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]\n\nGenerative AI has uses across a wide range of industries, including software

In [88]:
NUMBER=5 
SUBJECT="AI"
TONE="simple"
    

In [89]:
# Calling Chains

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed gener

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 17924 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}